In [1]:
# импорт библиотек,модулей
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row, Window
import pyspark.sql.types as T   
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from datetime import date
!pip install pandas
import pandas as pd
!pip install pyyaml ua-parser user-agents
from user_agents import parse

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# запуск spark сессии

spark = SparkSession.builder\
      .appName("Exem_work")\
      .config("spark.driver.bindAddress","localhost") \
      .config("spark.ui.port","4040") \
      .config("spark.driver.memory", "15g")\
      .getOrCreate()
spark.sparkContext.setLogLevel('WARN')

23/01/17 21:14:58 WARN Utils: Your hostname, data-analysis resolves to a loopback address: 127.0.1.1; using 146.120.224.166 instead (on interface ens160)
23/01/17 21:14:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/17 21:14:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# cоздаем схему будущего фрейма данных для LOG файла

sxemaL = T.StructType(fields=[
    T.StructField("client_ip", T.StringType(), True),
    T.StructField("type1", T.StringType(), True),
    T.StructField("type2", T.StringType(), True),
    T.StructField("dateO", T.StringType(), True),
    T.StructField("dateO1", T.StringType(), True),
    T.StructField("act", T.StringType (), True),
    T.StructField("status", T.IntegerType (), True),
    T.StructField("size", T.IntegerType (), True),
    T.StructField("type3", T.StringType (), True),
    T.StructField("user_agent", T.StringType (), True),
    T.StructField("type4", T.StringType (), True)
])

In [4]:
# LOG файл 

LOG_f = "access.log"
dfL = spark.read.csv(LOG_f, schema=sxemaL, sep=" ")
dfL.show(20)

+-------------+-----+-----+--------------------+------+--------------------+------+-----+--------------------+--------------------+-----+
|    client_ip|type1|type2|               dateO|dateO1|                 act|status| size|               type3|          user_agent|type4|
+-------------+-----+-----+--------------------+------+--------------------+------+-----+--------------------+--------------------+-----+
| 54.36.149.41|    -|    -|[22/Jan/2019:03:5...|+0330]|GET /filter/27|13...|   200|30577|                   -|Mozilla/5.0 (comp...|    -|
|  31.56.96.51|    -|    -|[22/Jan/2019:03:5...|+0330]|GET /image/60844/...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|    -|
|  31.56.96.51|    -|    -|[22/Jan/2019:03:5...|+0330]|GET /image/61474/...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|    -|
|40.77.167.129|    -|    -|[22/Jan/2019:03:5...|+0330]|GET /image/14925/...|   200| 1696|                   -|Mozilla/5.0 (comp...|    -|
|  91.99.72.15|    -|    -|[22/Jan

In [5]:
# проверяем значения по столбцу client_ip

dfL.groupby("client_ip")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = True)\
   .show()


+--------------+-----+
|     client_ip|activ|
+--------------+-----+
|  5.218.93.230|    1|
|   2.185.49.67|    1|
|  5.122.114.53|    1|
|   5.209.8.195|    1|
| 92.114.74.215|    1|
|   94.24.82.90|    1|
|   5.114.34.49|    1|
|   5.125.15.32|    1|
|158.58.127.196|    1|
|89.199.164.111|    1|
|   5.212.88.52|    1|
|  5.213.27.116|    1|
|  5.121.145.67|    1|
|   5.127.24.27|    1|
|  5.219.195.80|    1|
|  5.119.164.31|    1|
|   5.123.70.37|    1|
|   5.125.46.18|    1|
|  31.56.197.61|    1|
|113.203.11.180|    1|
+--------------+-----+
only showing top 20 rows



In [6]:
 dfL.groupby("client_ip")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|  10365152|
+----------+



In [7]:
# убираем столбцы которые не несут в себе полезной информации

df1 = dfL.drop("type1", "type2", "dateO1", "type3", "type4")
df1.show(20)

+-------------+--------------------+--------------------+------+-----+--------------------+
|    client_ip|               dateO|                 act|status| size|          user_agent|
+-------------+--------------------+--------------------+------+-----+--------------------+
| 54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/60844/...|   200| 5667|Mozilla/5.0 (Linu...|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/61474/...|   200| 5379|Mozilla/5.0 (Linu...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/14925/...|   200| 1696|Mozilla/5.0 (comp...|
|  91.99.72.15|[22/Jan/2019:03:5...|GET /product/3189...|   200|41483|Mozilla/5.0 (Wind...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/23488/...|   200| 2654|Mozilla/5.0 (comp...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/45437/...|   200| 3688|Mozilla/5.0 (comp...|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/576/ar...|   200|14776|Mozilla/5.

In [8]:
df1.count()

10365152

In [9]:
df1.printSchema()

root
 |-- client_ip: string (nullable = true)
 |-- dateO: string (nullable = true)
 |-- act: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- size: integer (nullable = true)
 |-- user_agent: string (nullable = true)



In [10]:
# т.к. client_ip не является первичным ключом и нам необходимо его создать самим. 
# Для этого добавим столбец log_id с нумерацией строк.

In [11]:
df11 = df1.withColumn("log_id", monotonically_increasing_id())

In [12]:
df11.show()

+-------------+--------------------+--------------------+------+-----+--------------------+------+
|    client_ip|               dateO|                 act|status| size|          user_agent|log_id|
+-------------+--------------------+--------------------+------+-----+--------------------+------+
| 54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|     0|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/60844/...|   200| 5667|Mozilla/5.0 (Linu...|     1|
|  31.56.96.51|[22/Jan/2019:03:5...|GET /image/61474/...|   200| 5379|Mozilla/5.0 (Linu...|     2|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/14925/...|   200| 1696|Mozilla/5.0 (comp...|     3|
|  91.99.72.15|[22/Jan/2019:03:5...|GET /product/3189...|   200|41483|Mozilla/5.0 (Wind...|     4|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/23488/...|   200| 2654|Mozilla/5.0 (comp...|     5|
|40.77.167.129|[22/Jan/2019:03:5...|GET /image/45437/...|   200| 3688|Mozilla/5.0 (comp...|     6|
|40.77.167

In [13]:
# смотрим формат дат и приводим к виду Y-M-D
df11.select("dateO").limit(2).show(truncate = 100)

+---------------------+
|                dateO|
+---------------------+
|[22/Jan/2019:03:56:14|
|[22/Jan/2019:03:56:16|
+---------------------+



In [14]:
from dateutil import parser

@F.udf(T.DateType())
def change_date_format(x):
    return parser.parse(x.replace(":", " ", 1))

In [15]:
 df2 = df11.select(F.regexp_replace(F.col("dateO"), "\[", "").alias("date"),"log_id" )\
          .select(change_date_format(F.col('date')).alias("date"), "log_id")

In [16]:
df2.show(truncate = 100)

+----------+------+
|      date|log_id|
+----------+------+
|2019-01-22|     0|
|2019-01-22|     1|
|2019-01-22|     2|
|2019-01-22|     3|
|2019-01-22|     4|
|2019-01-22|     5|
|2019-01-22|     6|
|2019-01-22|     7|
|2019-01-22|     8|
|2019-01-22|     9|
|2019-01-22|    10|
|2019-01-22|    11|
|2019-01-22|    12|
|2019-01-22|    13|
|2019-01-22|    14|
|2019-01-22|    15|
|2019-01-22|    16|
|2019-01-22|    17|
|2019-01-22|    18|
|2019-01-22|    19|
+----------+------+
only showing top 20 rows



In [17]:
df2.count()

10365152

In [18]:
# объединим таблицы

df_all = df2.join(df11, on='log_id', how = "left")
df_all.show(100)

+------------+----------+---------------+--------------------+--------------------+------+------+--------------------+
|      log_id|      date|      client_ip|               dateO|                 act|status|  size|          user_agent|
+------------+----------+---------------+--------------------+--------------------+------+------+--------------------+
|           0|2019-01-22|   54.36.149.41|[22/Jan/2019:03:5...|GET /filter/27|13...|   200| 30577|Mozilla/5.0 (comp...|
|           7|2019-01-22|  40.77.167.129|[22/Jan/2019:03:5...|GET /image/576/ar...|   200| 14776|Mozilla/5.0 (comp...|
|          19|2019-01-22|  66.249.66.194|[22/Jan/2019:03:5...|GET /m/filter/b2,...|   200| 19451|Mozilla/5.0 (Linu...|
|          22|2019-01-22|   66.249.66.91|[22/Jan/2019:03:5...|GET /filter/b874%...|   200| 39660|Mozilla/5.0 (comp...|
|          26|2019-01-22|    5.78.198.52|[22/Jan/2019:03:5...|GET /m/product/33...|   200| 21931|Mozilla/5.0 (Linu...|
|          29|2019-01-22|    91.99.72.15|[22/Jan

In [19]:
df_all.count()

10365152

In [20]:
df_all.printSchema()

root
 |-- log_id: long (nullable = false)
 |-- date: date (nullable = true)
 |-- client_ip: string (nullable = true)
 |-- dateO: string (nullable = true)
 |-- act: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- size: integer (nullable = true)
 |-- user_agent: string (nullable = true)



In [21]:
# убираем столбец dateO

df_all1 = df_all.drop("dateO")
df_all1.show(20)

+-----------+----------+---------------+--------------------+------+-----+--------------------+
|     log_id|      date|      client_ip|                 act|status| size|          user_agent|
+-----------+----------+---------------+--------------------+------+-----+--------------------+
|          0|2019-01-22|   54.36.149.41|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|
|          7|2019-01-22|  40.77.167.129|GET /image/576/ar...|   200|14776|Mozilla/5.0 (comp...|
|         19|2019-01-22|  66.249.66.194|GET /m/filter/b2,...|   200|19451|Mozilla/5.0 (Linu...|
| 8589934592|2019-01-22|    37.148.70.6|GET /image/1218/m...|   200|91935|Mozilla/5.0 (Wind...|
| 8589934596|2019-01-22|  66.160.188.37|GET /image/56/pro...|   200| 9298|Mozilla/5.0 (Linu...|
| 8589934599|2019-01-22|178.131.243.127|GET /image/64104/...|   200| 3698|Mozilla/5.0 (Wind...|
|17179869191|2019-01-22|185.120.137.243|GET /image/1540/p...|   200| 1988|Mozilla/5.0 (Wind...|
|17179869201|2019-01-22|    5.127.91.40|

In [22]:
df_all1.select("user_agent").count()

10365152

In [23]:
# смотрим вид данных для столбца user_agent

df_all1.select("user_agent").limit(10).show(truncate = 200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                              user_agent|
+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                      Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)|
|                                                                                 Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)|
|                                                                                       Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20100101 Firefox/64.0|
|                                    Mozilla/5.0 (Linux; Android 5.1.1

In [24]:
df3 = df11.select("user_agent", "client_ip", "log_id")\
.withColumn("device"
    , F.when(F.instr(F.col("user_agent"),"(compatible;") > 0
    , F.when(F.instr(F.col("user_agent"),"bingbot") > 0, "bingbot").otherwise(F.when(F.instr(F.col("user_agent"),"Googlebot") > 0, "Googlebot").otherwise("")))\
.otherwise(F.when(F.instr(F.col("user_agent"),"Windows") > 0, "Windows").otherwise(F.when(F.instr(F.col("user_agent"),"iPhone") + F.instr(F.col("user_agent"),"iPad") > 0, "Apple")\
    .otherwise(F.when(F.instr(F.col("user_agent"),"Android ") > 0, "Android").otherwise("")))))\
.withColumn("browser"\
    , F.when(F.instr(F.col("user_agent"),"(compatible;") > 0, "Spider")\
    .otherwise(F.when(F.instr(F.col("user_agent"),"Chrome/") > 0, "Chrome")\
        .otherwise(F.when(F.instr(F.col("user_agent"),"Firefox") > 0, "Firefox")\
            .otherwise(F.when(F.instr(F.col("user_agent"),"Safari/") > 0, "Mobile Safari").otherwise("")))))

df3.groupBy("device", "browser").count().orderBy("count", ascending=0).show(50,0)

+---------+-------------+-------+
|device   |browser      |count  |
+---------+-------------+-------+
|Windows  |Chrome       |2852581|
|Android  |Chrome       |2751333|
|Windows  |Firefox      |2092220|
|Googlebot|Spider       |642005 |
|Apple    |Mobile Safari|566330 |
|Android  |             |210059 |
|         |             |206949 |
|bingbot  |Spider       |200700 |
|Android  |Mobile Safari|156893 |
|Android  |Firefox      |156876 |
|         |Spider       |156420 |
|Windows  |             |136929 |
|         |Mobile Safari|133913 |
|         |Chrome       |78320  |
|         |Firefox      |17578  |
|Apple    |             |3308   |
|Windows  |Mobile Safari|2738   |
+---------+-------------+-------+



In [25]:
df3.show(10)

+--------------------+-------------+------+---------+-------+
|          user_agent|    client_ip|log_id|   device|browser|
+--------------------+-------------+------+---------+-------+
|Mozilla/5.0 (comp...| 54.36.149.41|     0|         | Spider|
|Mozilla/5.0 (Linu...|  31.56.96.51|     1|  Android| Chrome|
|Mozilla/5.0 (Linu...|  31.56.96.51|     2|  Android| Chrome|
|Mozilla/5.0 (comp...|40.77.167.129|     3|  bingbot| Spider|
|Mozilla/5.0 (Wind...|  91.99.72.15|     4|  Windows|Firefox|
|Mozilla/5.0 (comp...|40.77.167.129|     5|  bingbot| Spider|
|Mozilla/5.0 (comp...|40.77.167.129|     6|  bingbot| Spider|
|Mozilla/5.0 (comp...|40.77.167.129|     7|  bingbot| Spider|
|Mozilla/5.0 (comp...|66.249.66.194|     8|Googlebot| Spider|
|Mozilla/5.0 (comp...|40.77.167.129|     9|  bingbot| Spider|
+--------------------+-------------+------+---------+-------+
only showing top 10 rows



In [26]:
# убираем столбцы user_agent, client_ip

df3 = df3.drop("user_agent", "client_ip")
df3.show(20)

+------+---------+-------+
|log_id|   device|browser|
+------+---------+-------+
|     0|         | Spider|
|     1|  Android| Chrome|
|     2|  Android| Chrome|
|     3|  bingbot| Spider|
|     4|  Windows|Firefox|
|     5|  bingbot| Spider|
|     6|  bingbot| Spider|
|     7|  bingbot| Spider|
|     8|Googlebot| Spider|
|     9|  bingbot| Spider|
|    10|  bingbot| Spider|
|    11|  bingbot| Spider|
|    12|  Android| Chrome|
|    13|  bingbot| Spider|
|    14|         | Chrome|
|    15|  bingbot| Spider|
|    16|  bingbot| Spider|
|    17|  bingbot| Spider|
|    18|  bingbot| Spider|
|    19|Googlebot| Spider|
+------+---------+-------+
only showing top 20 rows



In [27]:
df3.select("log_id").count()

10365152

In [28]:
# объединим таблицы и получим базовую.

df_alls = df_all1.join(df3, on='log_id', how = "right")
df_alls.show(10)

+-----------+----------+---------------+--------------------+------+-----+--------------------+-------+-------+
|     log_id|      date|      client_ip|                 act|status| size|          user_agent| device|browser|
+-----------+----------+---------------+--------------------+------+-----+--------------------+-------+-------+
|          0|2019-01-22|   54.36.149.41|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|       | Spider|
|          7|2019-01-22|  40.77.167.129|GET /image/576/ar...|   200|14776|Mozilla/5.0 (comp...|bingbot| Spider|
| 8589934592|2019-01-22|    37.148.70.6|GET /image/1218/m...|   200|91935|Mozilla/5.0 (Wind...|Windows|Firefox|
| 8589934596|2019-01-22|  66.160.188.37|GET /image/56/pro...|   200| 9298|Mozilla/5.0 (Linu...|Android| Chrome|
| 8589934599|2019-01-22|178.131.243.127|GET /image/64104/...|   200| 3698|Mozilla/5.0 (Wind...|Windows| Chrome|
|17179869191|2019-01-22|185.120.137.243|GET /image/1540/p...|   200| 1988|Mozilla/5.0 (Wind...|Windows| 

In [29]:
df_alls.select('log_id').count()

10365152

In [30]:
# CSV файл с HDFS

CSV_f = spark.read.csv('hdfs://localhost:9099/user/tuhvatullina_gulgina/client_hostname.csv', header=True)

In [31]:
CSV_f.show(10)

+-------------+--------------------+--------------------+-----------------+
|       client|            hostname|          alias_list|     address_list|
+-------------+--------------------+--------------------+-----------------+
| 5.123.144.95|        5.123.144.95|[Errno 1] Unknown...|             null|
| 5.122.76.187|        5.122.76.187|[Errno 1] Unknown...|             null|
| 5.215.249.99|        5.215.249.99|[Errno 1] Unknown...|             null|
|31.56.102.211|31-56-102-211.sha...|['211.102.56.31.i...|['31.56.102.211']|
|5.123.166.223|       5.123.166.223|[Errno 1] Unknown...|             null|
|  5.160.26.98|         5.160.26.98|[Errno 1] Unknown...|             null|
|5.127.147.132|       5.127.147.132|[Errno 1] Unknown...|             null|
|158.58.30.218|       158.58.30.218|[Errno 1] Unknown...|             null|
| 86.55.230.86|        86.55.230.86|[Errno 1] Unknown...|             null|
| 89.35.65.186|        89.35.65.186|[Errno 1] Unknown...|             null|
+-----------

In [32]:
# проверяем содержимое столбцов CSV файла

CSV_f.count()

258445

In [33]:
CSV_f.groupby("client")\
   .agg(F.count("*").alias("activ"))\
   .orderBy("activ", ascending = True)\
   .show()

+--------------+-----+
|        client|activ|
+--------------+-----+
| 5.114.245.223|    1|
|92.223.105.186|    1|
|   5.116.31.39|    1|
|188.210.156.30|    1|
| 37.27.169.160|    1|
|  5.208.234.89|    1|
|   5.114.76.74|    1|
| 37.148.97.129|    1|
| 37.129.53.194|    1|
|  2.183.45.201|    1|
|  2.178.86.101|    1|
| 5.211.253.102|    1|
|  5.209.25.236|    1|
|  5.113.239.82|    1|
|  93.114.25.19|    1|
|195.181.93.133|    1|
|   5.75.72.167|    1|
| 2.182.233.205|    1|
|  5.112.211.99|    1|
| 5.117.104.154|    1|
+--------------+-----+
only showing top 20 rows



In [34]:
CSV_f.groupby("client")\
    .agg(F.count("*").alias("activ"))\
    .orderBy("activ", ascending = False)\
    .agg(F.sum("activ"))\
    .show()

+----------+
|sum(activ)|
+----------+
|    258445|
+----------+



In [35]:
# базовая таблица

df_alls.show(10)

+-----------+----------+---------------+--------------------+------+-----+--------------------+-------+-------+
|     log_id|      date|      client_ip|                 act|status| size|          user_agent| device|browser|
+-----------+----------+---------------+--------------------+------+-----+--------------------+-------+-------+
|          0|2019-01-22|   54.36.149.41|GET /filter/27|13...|   200|30577|Mozilla/5.0 (comp...|       | Spider|
|          7|2019-01-22|  40.77.167.129|GET /image/576/ar...|   200|14776|Mozilla/5.0 (comp...|bingbot| Spider|
| 8589934592|2019-01-22|    37.148.70.6|GET /image/1218/m...|   200|91935|Mozilla/5.0 (Wind...|Windows|Firefox|
| 8589934596|2019-01-22|  66.160.188.37|GET /image/56/pro...|   200| 9298|Mozilla/5.0 (Linu...|Android| Chrome|
| 8589934599|2019-01-22|178.131.243.127|GET /image/64104/...|   200| 3698|Mozilla/5.0 (Wind...|Windows| Chrome|
|17179869191|2019-01-22|185.120.137.243|GET /image/1540/p...|   200| 1988|Mozilla/5.0 (Wind...|Windows| 

In [36]:
df_alls.select("client_ip").count()

10365152

In [38]:
# выгружаем DataFrame в CSV файл

df_alls.write.csv("result.csv")

In [39]:
# суррогатный ключ - log_id

In [40]:
# выводим топ-5 самых активных посетителей сайта

df_alls.groupby("client_ip", "device").count().orderBy("count",ascending=False).show(5)

+---------------+---------+------+
|      client_ip|   device| count|
+---------------+---------+------+
|  66.249.66.194|Googlebot|346580|
|   66.249.66.91|Googlebot|205125|
|   66.249.66.91|         |109397|
|151.239.241.163|  Windows| 92448|
|   66.249.66.92|Googlebot| 56849|
+---------------+---------+------+
only showing top 5 rows



In [41]:
# выводим количество совершенных действий для данного устройства

df_client_device = df_alls.groupby('device')\
             .agg(F.count('client_ip').alias('count_u'))\
             .withColumn('count_u',F.col('count_u'))
             
df_client_device.show()

+---------+-------+
|   device|count_u|
+---------+-------+
|Googlebot| 642005|
|  Android|3275161|
|    Apple| 569638|
|         | 593180|
|  Windows|5084468|
|  bingbot| 200700|
+---------+-------+



In [42]:
# уберем нулевые значения в device

In [43]:
df_CD = df_client_device.filter(df_client_device.device != "")
df_CD.show()

+---------+-------+
|   device|count_u|
+---------+-------+
|Googlebot| 642005|
|  Android|3275161|
|    Apple| 569638|
|  Windows|5084468|
|  bingbot| 200700|
+---------+-------+



In [44]:
# посчитаем долю пользователей данного устройства от общего числа пользователей

In [45]:
df_device_client = df_CD.withColumn('all_users', F.sum('count_u').over(Window.partitionBy()))\
            .withColumn('ratio', F.col('count_u') / F.col('all_users'))\
            .select('device', 'count_u', 'ratio')
       
df_device_client.show()

23/01/17 21:21:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:21:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:21:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:21:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:21:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:21:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+---------+-------+--------------------+
|   device|count_u|               ratio|
+---------+-------+--------------------+
|Googlebot| 642005| 0.06569861231694074|
|  Android|3275161| 0.33515865579639403|
|    Apple| 569638| 0.05829304463827772|
|  Windows|5084468|  0.5203113557836637|
|  bingbot| 200700|0.020538331464723805|
+---------+-

In [46]:
# выводим топ-5 самых популярных браузеров

df0 = df_alls.groupby("browser").count()
df0.show(5)

+-------------+-------+
|      browser|  count|
+-------------+-------+
|       Spider| 999125|
|      Firefox|2266674|
|Mobile Safari| 859874|
|       Chrome|5682234|
|             | 557245|
+-------------+-------+



In [49]:
# посчитаем долю 5 самых популярных браузера от общего числа браузеров

In [50]:
df_browser_client = df0.withColumn('all_users', F.sum('count').over(Window.partitionBy()))\
            .withColumn('ratio', F.col('count') / F.col('all_users'))\
            .select('browser', 'count', 'ratio')
       
df_browser_client.show(5)

23/01/17 21:22:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:22:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:22:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/01/17 21:22:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/17 21:22:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+-------------+-------+-------------------+
|      browser|  count|              ratio|
+-------------+-------+-------------------+
|       Spider| 999125| 0.0963927012358333|
|      Firefox|2266674|0.21868217658554356|
|Mobile Safari| 859874| 0.0829581659776914|
|       Chrome|5682234| 0.5482055641827539|
|             | 557245|0.0537613